In [1]:
import os, sys, requests, tarfile, re, time

In [2]:
def check_file_status(filepath, filesize):
    sys.stdout.write('\r')
    sys.stdout.flush()
    size = int(os.stat(filepath).st_size)
    percent_complete = (size/filesize)*100
    sys.stdout.write('%.3f %s' % (percent_complete, '% Completed'))
    sys.stdout.flush()

with open('/media/sf_D_DRIVE/rdaPass.txt', 'r') as file:
    pswd = file.read().replace('\n', '')

url = 'https://rda.ucar.edu/cgi-bin/login'
values = {'email' : 'scottaburgholzer@lewisu.edu', 'passwd' : pswd, 'action' : 'login'}
# Authenticate
ret = requests.post(url,data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

In [3]:
def getData(dspath, filelist):
    for file in filelist:
        
        # Download the file
        filename=dspath+file
        file_base = os.path.basename(file)
        
        print('Downloading',file_base)
        req = requests.get(filename, cookies = ret.cookies, allow_redirects=True, stream=True)
        filesize = int(req.headers['Content-length'])
        with open(file_base, 'wb') as outfile:
            chunk_size=1048576
            for chunk in req.iter_content(chunk_size=chunk_size):
                outfile.write(chunk)
                if chunk_size < filesize:
                    check_file_status(file_base, filesize)
        check_file_status(file_base, filesize)
        
        # extract the file
        tf = tarfile.open(file_base)
        tf.extractall()
        tf.close()
        
        # wait 15 seconds
        time.sleep(15)
        
        print("Extracting Data....")
        # get the 12Z file only
        fileSplit = file_base.split(".") 
        gribName = fileSplit[0] + "." + fileSplit[1] + "12"
        
        # delete all files not needed
        gribFiles = [f for f in os.listdir('/media/sf_D_DRIVE/MSDS-Capstone-Project/notebooks') if re.match(r'merged_AWIP.*',f)]
        for file in gribFiles:
            if file != gribName:
                os.remove("/media/sf_D_DRIVE/MSDS-Capstone-Project/notebooks/" + file)
        
        # Create the CTL file
        os.system('perl /media/sf_D_DRIVE/MSDS-Capstone-Project/notebooks/grib2ctl.pl -verf /media/sf_D_DRIVE/MSDS-Capstone-Project/notebooks/' + gribName + '>' + gribName + '.ctl')
        # Create the IDX file
        os.system('gribmap -v -i ' + gribName + '.ctl')
        
        # run the GrADS script
        os.system('grads -bxcl "run /media/sf_D_DRIVE/MSDS-Capstone-Project/notebooks/gradsScripts/arealAverages.gs /media/sf_D_DRIVE/MSDS-Capstone-Project/notebooks/' + gribName + '.ctl"')
        print()

In [ ]:
dspath = 'https://rda.ucar.edu/dsrqst/BURGHOLZER395399/'
filelist = [
'merged_AWIP32.19790101.tar',
]

getData(dspath, filelist)

100.000 % CompletedExtracting Data....

100.000 % Completed

In [2]:
#

0

In [24]:
print('gribmap -v -i ' + gribName + '.ctl')

NameError: name 'gribName' is not defined